In [1]:
import trimesh

from fidget.types import GearConfig, ShapeConfig, ShapeType, PinConfig, PrintLayoutConfig
from fidget.ops import build_core, build_gear, align_gear, rotate_gear, build_shape, apply_shape, build_pin, add_holes, add_hole, align_gears_vertically, arrange_meshes


rotation_angles = [27, 17, 7, -3, -3, 7, 17, 27]
CORE_DIAMETER = 35.0

core = build_core(diameter=CORE_DIAMETER)
gear_config = GearConfig(profile="./assets/gear.svg", base_diameter=24.0, height=500.0, angle=5.7)

def process_gear(gear_config, face, angle):
    gear = build_gear(gear_config)
    aligned_gear = align_gear(gear, face, 0)
    return rotate_gear(aligned_gear, face, angle)

gears = [process_gear(gear_config, face, angle) 
         for angle, face in zip(rotation_angles, core.faces)]


# shape_config = ShapeConfig(
#     shape_type=ShapeType.CUBE, 
#     size=[40.0, 40.0, 40.0], 
#     scale=(1.0, 1.0, 1.0), 
#     rotation=(0, 0, 0), 
#     offset=(0, 0, 0)
# )

shape_config = ShapeConfig(
    shape_type=ShapeType.CUSTOM, 
    stl_path="./assets/hyperboloid.stl", 
    rotation=(0, 0, 0), 
    offset=(0, 0, 0), 
    scale=(3, 3, 3)
)
shape = build_shape(shape_config)

gears_cut_to_shape = apply_shape(shape, gears)

pin = build_pin(PinConfig(diameter=6.0, length=12.0, center_length=2.0))
core_with_holes = add_holes(core, pin, 1.05)

gears_cut_to_shape_with_holes = [add_hole(gear, face, pin, 1.05) for gear, face in zip(gears_cut_to_shape, core.faces)]

scene = trimesh.Scene([core_with_holes, *gears_cut_to_shape_with_holes])
scene.show()

In [2]:
arranged_gears = arrange_meshes(align_gears_vertically(gears_cut_to_shape_with_holes, core.faces), PrintLayoutConfig(row_spacing=35, col_spacing=35, rows=3, cols=3))
arranged_pins = arrange_meshes([pin.pin_mesh.copy() for _ in range(8)], PrintLayoutConfig(row_spacing=23, col_spacing=10, rows=3, cols=3))


def save_stl():
    from pathlib import Path
    
    # Create output directory if it doesn't exist
    output_dir = Path("./output")
    output_dir.mkdir(exist_ok=True)
    
    # Save files to output directory
    trimesh.Scene(arranged_pins).export(output_dir / "pins.stl")
    for i, gear in enumerate(arranged_gears):
        gear.export(output_dir / f"gear_{i}.stl")
    core_with_holes.mesh.export(output_dir / "core.stl")

save_stl()